In [1]:
import cv2
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.models import load_model
%matplotlib inline

In [12]:
#declaration of global variables
#initialize skeleton models and variables for individual frame skeleton detection
protoFile = "../MPI/pose_deploy_linevec.prototxt"
weightsFile = "../MPI/pose_iter_160000.caffemodel"
#load pretrained models for single person joint detection
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
jointCount0 = 15
POSE_PAIRS =[ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

#directories list of dataset for each class
category = [cat for cat in os.listdir('../dataset')]
'''mutliple dancers joint keypoint extraction weight and network model trained on coco'''

#load weight and network files
cocoProtoFile = "../coco/pose_deploy_linevec.prototxt"
cocoCaffeWeightsFile = "../coco/pose_iter_440000.caffemodel"
jointCount = 18

#initialize skeleton pair nodes and labels based on coco training dataset
keypointPairs = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],[1,8], [8,9], [9,10], [1,11], [11,12], [12,13],[1,0], [0,14], [14,16], [0,15], [15,17],[2,17], [5,16] ]
keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip','L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']
keypointPairsIndex = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44],[19,20], [21,22], [23,24], [25,26], [27,28], [29,30], [47,48], [49,50], [53,54], [51,52], [55,56], [37,38], [45,46]]
keypointPairsColor = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],[0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],[0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]
#initialize the pre-trained coco network using cv dnn, weight from caffe file, network from protofile
cocoDnn = cv2.dnn.readNetFromCaffe(cocoProtoFile, cocoCaffeWeightsFile)

# Output labels to to classify dance sequence
labels = ["gojjam","guragingna","shoa"]
#modelFile
modelFile = "../models/sample.h5"
sourceFile = "../test/test1.mkv"

In [13]:
#real time emotion prediction from specified source
def realPred(source,model,labels):
    liveFrame = cv2.VideoCapture(source)
    faceDetectioin = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    sequence = 0
    label = ""
    predicted = False
    curentFrameSequenceKeypointList = []
    while True:

        val, img = liveFrame.read()
        
        if not val:
            continue;
        # resize video frames
        frame = cv2.resize(img, (640, 360))
        
        if(sequence < 20 and not predicted):
            sequence+=1
            currentKeypoints = skeletonizeDancePose(frame,0.1)
            curentFrameSequenceKeypointList.append(np.array(currentKeypoints).astype(np.float32))
            continue
        
        else:
            
            data = (np.array(curentFrameSequenceKeypointList).astype(np.float32))
            dancerII = [] 
            #print(len(curentFrameSequenceKeypointList))
            for k in range(len(data)):

                currentStep = np.copy(data[k])
                currentStep = currentStep.flatten()
                dancerII.append(np.array(currentStep).astype(np.float32))
            sequenceData = np.array(dancerII).astype(np.float32)
            sequenceData = sequenceData[np.newaxis, :, :]
            if(not predicted):
                
                result=model.predict(sequenceData,None)
                labelClass= result.argmax(axis=-1)
                label=labels[labelClass[0]]
                print(" Label :",label)
                predicted = True
                return

        testImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        detectedFaces = faceDetectioin.detectMultiScale(testImg, 1.32, 5)

        for (x,y,w,h) in detectedFaces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,128,0),thickness=4)

            cv2.putText(img, label, (int(x), int(y)), cv2.FONT_HERSHEY_DUPLEX, 1, (120,0,255), 2)

        resized_img = cv2.resize(img, (640, 360))
        cv2.imshow('Analayzing Dance Form',resized_img)

        if cv2.waitKey(10) & 0xFF == ord('e'):
            break

    liveFrame.release()
    cv2.destroyAllWindows

#for single person skeleton extraction
def skeletonizeDancePose(img, thresh):

    frameWidth = img.shape[1]
    frameHeight = img.shape[0]
    frameCopy = np.copy(img)
    threshold = 0.1
    
    #preprocess img for input
    inpBlob = cv2.dnn.blobFromImage(img, 1.0 / 255, (368, 368),(0, 0, 0), swapRB=False, crop=False)
    #feed the preprocessed img through the network
    cocoDnn.setInput(inpBlob)
    output = cocoDnn.forward()
    H = output.shape[2]
    W = output.shape[3]
    
    # gather keypoints here
    points = []
    
    #loop through possible 18 keypoints
    for i in range(jointCount):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # Scale the point to fit on the original image
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        if prob > thresh :
            cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
            cv2.circle(img, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))
        else :
            points.append((0,0))
        
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(img, points[partA], points[partB], (0, 255, 255), 3)
    
    return np.array(points)
    plt.figure(figsize=[10,10])
    plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
    plt.figure(figsize=[10,10])
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #cv2.imshow("Subtraction result", frame)
    
    #return the curent 18 body part point pose
 

In [14]:
#Load the action-recognition trained model
model=load_model(modelFile)
data2 = realPred(sourceFile,model,labels)


 Label : shoa


In [15]:
data2.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [47]:
dancerII = [] 
#print(len(curentFrameSequenceKeypointList))
for k in range(len(data2)):
                            
    currentStep = np.copy(data2[k])
    currentStep = currentStep.flatten()
    dancerII.append(np.array(currentStep).astype(np.float32))


In [48]:
sequence = np.array(dancerII).astype(np.float32)

In [50]:
sequence = sequence[np.newaxis, :, :]

In [51]:
sequence.shape

(1, 5, 36)